In [10]:
# Import necessary libraries for numerical operations, plotting, TensorFlow, and image processing
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import preprocess_input # Specific preprocessing for VGG16
from tensorflow.keras.optimizers import Adam
from keras import callbacks, layers, models, optimizers, regularizers # Keras components for model building
import os # Operating system functionalities, e.g., path joining
import cv2 # OpenCV for image processing tasks
from keras.losses import SparseCategoricalCrossentropy # Loss function for multi-class classification
from PIL import Image # Python Imaging Library for image manipulation

In [6]:
# Define input shape for the images (height, width, channels)
input_shape = (64, 64, 3)
# Define batch size for training and evaluation
batch_size = 64
# Define the number of epochs for training; EarlyStopping may halt training sooner
num_epochs = 50

In [7]:
# Define the path to the preprocessed RGB images directory
preprocessed_path = r"D:\FUCK!!\Pattern\Project\notebooks\preprocessed_RGB_images"

# Initialize ImageDataGenerator with VGG16 preprocessing function
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Setup the training data generator
print("Setting up Training Generator...")
train_generator = train_datagen.flow_from_directory(
    os.path.join(preprocessed_path, 'train'), # Path to the training data
    target_size=input_shape[:2], # Resize images to the defined input_shape
    batch_size=batch_size, # Set the batch size
    class_mode='sparse', # Use 'sparse' for sparse_categorical_crossentropy loss
    color_mode='rgb', # Ensure images are loaded in RGB format
    shuffle=True # Shuffle the training data
)
print(f"Found {train_generator.samples} images belonging to {train_generator.num_classes} classes in training set.")

# Setup the validation data generator
print("\nSetting up Validation Generator...")
val_generator = train_datagen.flow_from_directory(
    os.path.join(preprocessed_path, 'val'), # Path to the validation data
    target_size=input_shape[:2], # Resize images
    batch_size=batch_size, # Set batch size
    class_mode='sparse', # Use 'sparse' for loss function
    color_mode='rgb', # Load images in RGB
    shuffle=False # No need to shuffle validation data
)
print(f"Found {val_generator.samples} images belonging to {val_generator.num_classes} classes in validation set.")

# Setup the testing data generator
print("\nSetting up Testing Generator...")
test_generator = train_datagen.flow_from_directory(
    os.path.join(preprocessed_path, 'test'), # Path to the test data
    target_size=input_shape[:2], # Resize images
    batch_size=batch_size, # Set batch size
    class_mode='sparse', # Use 'sparse' for loss function
    color_mode='rgb', # Load images in RGB
    shuffle=False # No need to shuffle test data
)
print(f"Found {test_generator.samples} images belonging to {test_generator.num_classes} classes in validation set.")

Setting up Training Generator...
Found 30580 images belonging to 10 classes.
Found 30580 images belonging to 10 classes in training set.

Setting up Validation Generator...
Found 328 images belonging to 10 classes.
Found 328 images belonging to 10 classes in validation set.

Setting up Testing Generator...
Found 328 images belonging to 10 classes.
Found 328 images belonging to 10 classes in validation set.


In [22]:
# Define the CNN model architecture using Keras Sequential API
model = models.Sequential([
    layers.Input(shape=input_shape), # Define the input layer with the specified shape

    # Convolutional Block 1
    layers.Conv2D(32, 3, padding='same', activation='relu'), # 32 filters, 3x3 kernel, ReLU activation
    layers.BatchNormalization(), # Normalize activations
    layers.MaxPooling2D(2), # Max pooling with 2x2 pool size

    # Convolutional Block 2
    layers.Conv2D(64, 3, padding='same', activation='relu'), # 64 filters
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),

    # Convolutional Block 3
    layers.Conv2D(128, 3, padding='same', activation='relu'), # 128 filters
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),

    # Convolutional Block 4
    layers.Conv2D(256, 3, padding='same', activation='relu'), # 256 filters
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),

    # Convolutional Block 5
    layers.Conv2D(512, 3, padding='same', activation='relu'), # 512 filters
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),

    # Classification Head
    layers.GlobalAveragePooling2D(), # Global average pooling to flatten feature maps
    layers.Dense(512, activation='relu'), # Fully connected layer with 512 units
    layers.BatchNormalization(),
    layers.Dropout(0.5), # Dropout for regularization (50% dropout rate)

    layers.Dense(256, activation='relu'), # Fully connected layer with 256 units
    layers.BatchNormalization(),
    layers.Dropout(0.5), # Dropout for regularization
    layers.Dense(10, activation='softmax', name='predictions') # Output layer with 10 units (for 10 classes) and softmax activation
])

# Compile the model
# Use Adam optimizer with a specific, smaller learning rate for fine-tuning or stable training
optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy', # Loss function for integer labels
              metrics=['accuracy'])

# Print the model summary
print("\nModel Summary:")
model.summary()

In [11]:
# Define Callbacks for training
# Early stopping to prevent overfitting and stop training when validation loss stops improving
early_stopping = EarlyStopping(
    monitor='val_loss', # Metric to monitor
    patience=5, # Number of epochs with no improvement after which training will be stopped
    verbose=1, # Log when training is stopped
    restore_best_weights=True # Restore model weights from the epoch with the best validation loss
)

# Model checkpoint to save the best model found during training based on validation loss
model_checkpoint = ModelCheckpoint(
    filepath='best_CNN_model.keras', # File path to save the model
    monitor='val_loss', # Metric to monitor
    save_best_only=True, # Only save a model if `val_loss` has improved
    verbose=1 # Log when a model is saved
)

In [12]:
# Train the model using the training and validation generators
print(f"\nStarting training for up to {num_epochs} epochs (head only)...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size, # Number of steps per epoch
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size, # Number of validation steps
    epochs=num_epochs, # Maximum number of epochs
    callbacks=[early_stopping, model_checkpoint] # List of callbacks to use during training
)
print("\nTraining finished.")

In [13]:
# Load the best saved model for evaluation or further use
from tensorflow.keras.models import load_model
model = load_model(r'D:\FUCK!!\Pattern\Project\Models\best_CNN_model.keras')

In [15]:
# Import necessary libraries for model evaluation
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix

# Prepare a test data generator with only preprocessing (no augmentation)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_gen = test_datagen.flow_from_directory(
    directory=r'D:\FUCK!!\Pattern\Project\notebooks\preprocessed_RGB_images\test', # Path to the test dataset
    target_size=input_shape[:2], # Resize images to the model's input shape
    batch_size=batch_size, # Batch size for evaluation
    class_mode='sparse', # For integer labels
    shuffle=False # Important: Do not shuffle for evaluation to align predictions with true labels
)

Found 328 images belonging to 10 classes.


In [16]:
# Clear any old Keras session/graph to avoid conflicts
tf.keras.backend.clear_session()

# Reload the saved .keras model
model = load_model(r'D:\FUCK!!\Pattern\Project\Models\best_CNN_model.keras')

# Re-compile the model with the correct loss, metrics, and enable eager execution for evaluation
# Eager execution can sometimes help with debugging or specific evaluation scenarios
model.compile(
    optimizer=Adam(learning_rate=1e-4), # Adam optimizer with a small learning rate
    loss='sparse_categorical_crossentropy', # Loss function for integer labels
    metrics=['accuracy'], # Metric to evaluate
    run_eagerly=True # Disables tf.function wrapping, runs operations eagerly
)

# Evaluate the model on the test generator
loss, acc = model.evaluate(test_gen, verbose=1)
print(f"\nTest loss: {loss:.4f} — Test accuracy: {acc:.4f}")

In [17]:
# Obtain per-class metrics
# Calculate the number of steps needed to cover the entire test set
steps = int(np.ceil(test_gen.samples / batch_size))

# Predict class probabilities for the test set
pred_probs = model.predict(
    test_gen,
    steps = steps, # Ensure all test samples are predicted
    verbose=1 # Show prediction progress
)

# Convert predicted probabilities to class indices by taking the argmax
pred_idxs = np.argmax(pred_probs, axis=1)

# Get true class indices from the test generator
true_idxs = test_gen.classes

# Get the list of class labels (names)
labels = list(test_gen.class_indices.keys())

In [18]:
# Generate and print the classification report
print("\nClassification Report:\n")
print(classification_report(true_idxs, pred_idxs, target_names=labels))

In [19]:
# Generate and print the confusion matrix
print("\nConfusion Matrix:\n")
print(confusion_matrix(true_idxs, pred_idxs))

In [20]:
# Plot training & validation accuracy values
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(); 
plt.title('Training and Validation Accuracy')
plt.show()

In [21]:
# Plot training & validation loss values
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(); 
plt.title('Training and Validation Loss')
plt.show()
